### Simple motion play of `Common Rig` 

In [3]:
import pickle, os
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig.xml`

In [4]:
xml_path = '../asset/common_rig/scene_common_rig.xml'
env = MuJoCoParserClass(name='Common Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

<_MjModelActuatorViews
  acc0: array([1825.36])
  actadr: array([-1], dtype=int32)
  actlimited: array([0], dtype=uint8)
  actnum: array([0], dtype=int32)
  actrange: array([0., 0.])
  biasprm: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  biastype: array([0], dtype=int32)
  cranklength: array([0.])
  ctrllimited: array([1], dtype=uint8)
  ctrlrange: array([-1.,  1.])
  dynprm: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  dyntype: array([0], dtype=int32)
  forcelimited: array([0], dtype=uint8)
  forcerange: array([0., 0.])
  gainprm: array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  gaintype: array([0], dtype=int32)
  gear: array([200.,   0.,   0.,   0.,   0.,   0.])
  group: array([0], dtype=int32)
  id: 0
  length0: array([0.])
  lengthrange: array([0., 0.])
  name: 'root1'
  trnid: array([ 1, -1], dtype=int32)
  trntype: array([0], dtype=int32)
  user: array([], dtype=float64)
>
<_MjModelActuatorViews
  acc0: array([9121.79])
  actadr: array([-1], dtype=int32)
  actlimited:

In [5]:
# Set which joints to control
rev_joint_names = env.ctrl_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=rev_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=rev_joint_names)
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

### Load motion and playback

In [6]:
motion_file_path = os.path.join("../asset/common_rig/motion/walk.npy")
device='cpu'

In [10]:
# from amp.utils.motion_lib import MotionLib
from amp.tasks.common_rig_amp import MotionLib
motion_lib = MotionLib(motion_file=motion_file_path,
                       num_dofs=env.n_rev_joint,
                       key_body_ids=np.array([]),
                       device=device)
motion_lib.get_motion(0)
qs = motion_lib.get_motion(0)._q_pos
p_roots = motion_lib.get_motion(0).root_translation
# p_roots += 0.05
quat_roots = motion_lib.get_motion(0).global_root_rotation
L = qs.shape[0]

/Users/yoonbyung/opt/anaconda3/envs/snapbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-12 18:25:26,438	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-12 18:25:26,589 - INFO - logger - logger initialized
/Users/yoonbyung/Dev/yet-another-mujoco-tutorial-v3/code/amp/poselib/poselib/core/rotation3d.py:222: DeprecationWarning: invalid escape sequence \*
  """


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html
Loading 1/1 motion files: ../asset/common_rig/motion/walk.npy
Loaded 1 motions with a total length of 2.450s.


In [11]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
while env.is_viewer_alive():
    # Update 
    q = qs[tick,:] # [35]
    p_root = p_roots[tick,:] # [3]
    quat_root = quat_roots[tick,:] # [4] quaternion
    # tick = min(tick+1,L-1)
    if tick==(L-1): tick = 0
    else: tick = tick + 1
    env.set_p_root(root_name='base',p=p_root)
    env.set_quat_root(root_name='base',quat=quat_root[[3,0,1,2]])
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

: 

### First, get the sequence of two feet pose

In [10]:
p_rfoot_traj = np.zeros((L,3))
p_lfoot_traj = np.zeros((L,3))
for tick in range(L):
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    env.set_p_root(root_name='base',p=p_root)
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    # Append
    p_rfoot_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_traj[tick,:] = env.get_p_geom(geom_name='lfoot')

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [11]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
tick = 0
p_rfoot_centered_traj = np.zeros((L,3))
p_lfoot_centered_traj = np.zeros((L,3))
while env.is_viewer_alive():
    
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Append translated foot traj
    p_rfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='rfoot')
    p_lfoot_centered_traj[tick,:] = env.get_p_geom(geom_name='lfoot')
    
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

### Solve IK for two feet

In [ ]:
p_trgt_rfoot = np.average(p_rfoot_centered_traj,axis=0)
p_trgt_lfoot = np.average(p_lfoot_centered_traj,axis=0)
R_trgt_rfoot = rpy2r(np.radians([0,0,0]))
R_trgt_lfoot = rpy2r(np.radians([0,0,0]))
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()
while env.is_viewer_alive():
    # Update
    q = sample_qs[0,tick,:] # [35]
    p_root = sample_p_roots[0,tick,:] # [3]
    p_cfoot = 0.5*(p_rfoot_traj[tick,:]+p_lfoot_traj[tick,:])
    env.set_p_root(root_name='base',p=p_root-p_cfoot+np.array([0,0,0.02]))
    env.forward(q=q,joint_idxs=joint_idxs_fwd)
    
    # Solve IK
    ik_geom_names = ['rfoot','lfoot']
    ik_p_trgts = [p_trgt_rfoot,p_trgt_lfoot]
    ik_R_trgts = [R_trgt_rfoot,R_trgt_lfoot]
    err_traj = np.zeros(30)
    for ik_tick in range(30):
        J_list,ik_err_list = [],[]
        for ik_idx,ik_geom_name in enumerate(ik_geom_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            J,ik_err = env.get_ik_ingredients_geom(
                geom_name=ik_geom_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=True,IK_R=True)
            J_list.append(J)
            ik_err_list.append(ik_err)
        J_stack      = np.vstack(J_list)
        ik_err_stack = np.hstack(ik_err_list)
        err_traj[ik_tick] = np.max(np.abs(ik_err_stack))
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        env.forward(q=q,joint_idxs=joint_idxs_fwd)
        
    # Render
    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        # Plot foot
        env.plot_geom_T(geom_name='rfoot',axis_len=0.3)
        env.plot_geom_T(geom_name='lfoot',axis_len=0.3)
        # Plot revolute joints with arrow
        env.plot_joint_axis(axis_len=0.1,axis_r=0.01)    
        env.render()
        
    # Increase tick
    if tick==(L-1): tick = 0
    else: tick = tick + 1
# Close MuJoCo viewer
env.close_viewer()
print ("Done.")

Done.
